In [1]:
import os

PATH_DUUI = "data/duui-uima"
FILTER_FILES = {".java"}
count = 0
LIST_XML_FILES = []
for root, subdirs, files in os.walk(PATH_DUUI):
    for file in files:
        current_file = os.path.join(root, file)
        # filter files and ignore pom.xml
        if (file.endswith(tuple(FILTER_FILES)) and (file != "pom.xml")):
            LIST_XML_FILES.append(current_file)
print(len(LIST_XML_FILES))

155


In [2]:
import sys
print(sys.executable)

/usr/bin/python3


In [3]:
#import sys
#!"{sys.executable}" -m pip install -U chromadb

In [2]:
!pip install ollama

In [3]:
import chromadb as cdb
import ollama

client = cdb.PersistentClient("chroma")
collection = client.get_or_create_collection(name="JAVA_test")
print(collection.count)

response = ollama.embed(
                model='mxbai-embed-large',
                input="Wie mache ich aus einer liste von strings ein cas object"
                ).embeddings
print(len(response[0]))
print(collection.query(query_embeddings=response[0], n_results=1)["metadatas"])


ModuleNotFoundError: No module named 'ollama'

ModuleNotFoundError: No module named 'ollama'

In [6]:
# Testing multiple ways to Embed Code
# Load model directly


In [1]:
# chunk the files with chunker

from tqdm import tqdm
import chunker
import uuid
from openai import OpenAI


# cast files to Chroma format

count = 0
for file in tqdm(LIST_XML_FILES):
    chunks = chunker.chunk_python_file(file, include_header=True, include_methods=True)
    ids, metas, docs = [], [], []
    for chunk in chunks:
        chroma_format = chunker.chunk_to_chroma_item(chunk.text, chunk.meta)
        # some empty documents exist
        if chroma_format["document"] != "":           
            ids.append(chroma_format["id"])
            metas.append(chroma_format["metadata"])
            docs.append(chroma_format["document"])
        else:
            print("leerer eintrag")
    if ids:
        collection.add(ids=ids, metadatas=metas, documents=docs)
count

NameError: name 'LIST_XML_FILES' is not defined

In [1]:
collection.query(query_texts= "write the DUUIReponse class for my hate modell", n_results=5)


NameError: name 'collection' is not defined